In [9]:
from gensim.models import KeyedVectors
import sys
sys.path.append("../../imports/")
import saver as sv
import numpy as np

word2desc = sv.load("word2desc")

In [4]:
w2v = '/home/manni/embs/word2vec-google-news-300.gz'
model = KeyedVectors.load_word2vec_format(w2v, binary=True)

In [5]:
def sense_vec(word,model):
    '''
    Computes average vector from sense description. 
    Parameters
    ----------
    word : str
        Unicode or utf-8 encoded string.
    model : KeyedVectors
        gensim KeyedVectors object
    Returns
    -------
    list
        List of [Vector,Vector,....].
    '''
    dim = model.vector_size
    v = list()
    if word not in word2desc:
        return [np.zeros(dim)]
    for words in word2desc[word]:
        if not words:
            continue
        _v = list()
        for _word in words:
            if _word in model.vocab:
                _v.append(model.get_vector(_word))
        if not _v:
            continue
        if not _v:
            _v = [np.zeros(dim)]
        _v = np.sum(_v,axis=0)
        v.append(_v)
    return v


In [6]:
def tokenize(tokens,WINDOW,model):
    to_replace = dict()
    for i,token in enumerate(tokens):
        if token in word2desc:
            left = tokens[i-WINDOW:i]
            right = tokens[i+1:i+WINDOW+1]
            context = set(left+right)
            svecs = sense_vec(token,model)
            maxi = 0
            tag = -1 #index in wordsense
            for si,v in enumerate(svecs):
                for con in context:
                    if con == token:
                        continue
                    _svecs = sense_vec(con,model)
                    for sj,_v in enumerate(_svecs):
                        if np.sum(_v)==0:
                            continue
                        sim = model.cosine_similarities(v, [_v])[0]
                        if sim>maxi:
                            maxi = sim
                            tag = si
            if tag>=0:
                to_replace[i]=token+'#'+str(tag)

In [7]:
tokens = 'part river where current very fast'
tokens = tokens.split()

In [10]:
%timeit tokenize(tokens,3,model)

81.5 ms ± 974 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
